In [ ]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

ССылки на файлы

In [ ]:
ASU_dataset = Path('data/train_dataset_train.csv')

Загрузка баового датасета

In [ ]:

asu_frame = pd.read_csv(ASU_dataset)
asu_frame.head(2)

In [ ]:
asu_frame.shape

In [ ]:
asu_frame['one'] = 0
asu_frame['three'] = 0
asu_frame['four'] = 0
asu_frame.loc[asu_frame['Статус'] == -1, 'one'] = -1
asu_frame.loc[asu_frame['Статус'] == 3, 'three'] = 3
asu_frame.loc[asu_frame['Статус'] == 4, 'four'] = 4


Очистка значений датафрейма

In [ ]:
asu_frame.loc[asu_frame['Пол'] == 'муж', 'Пол'] = 'Муж'

In [ ]:
asu_frame['male'] = 0
asu_frame['female'] = 0

asu_frame.loc[asu_frame['Пол'] == 'Муж', 'male'] = 1
asu_frame.loc[asu_frame['Пол'] == 'Жен', 'female'] = 1
asu_frame['sex_summ'] = asu_frame['male'] + asu_frame['female']
asu_frame.loc[asu_frame['sex_summ'] != 1].shape[0]

In [ ]:
#asu_frame['birth_year'] = asu_frame['Дата_Рождения'].dt.to_period('Y')
asu_frame['Дата_Рождения']
asu_frame['birth_year'] = pd.to_datetime(asu_frame['Дата_Рождения'])
asu_frame['birth_year'].dt.to_period('Y')
asu_frame['birth_year'].all()

asu_frame['entrance'] = pd.to_datetime(asu_frame['Год_Поступления'], format='%Y')
asu_frame['entrance'].sort_values()



In [ ]:
asu_frame['birth_year_int'] = asu_frame['Дата_Рождения'].str.slice(start=0, stop=4).astype('int64')
asu_frame['birth_year_int']

In [ ]:
frame_size = asu_frame.shape
frame_size

In [ ]:
asu_frame.columns

In [ ]:
asu_frame['Статус']

In [ ]:
asu_frame.groupby('Статус')['Село'].value_counts()

In [ ]:
asu_frame.pivot_table(index='Пол', columns='Статус')

In [ ]:
graduate = asu_frame.loc[asu_frame['Статус'] == 4]['Статус'].count()
expelled = asu_frame.loc[asu_frame['Статус'] == 3]['Статус'].count()
vacation = asu_frame.loc[asu_frame['Статус'] == -1]['Статус'].count()
all_students = frame_size[0]
delta = all_students - graduate - expelled - vacation
f'Всего позиций {all_students} закончили ВУЗ {graduate}, отчислено {expelled}, каникулы {vacation}. Отклонение по сумме {delta}'

In [ ]:
graduate_frame = asu_frame.loc[asu_frame['Статус'] == 4]
graduate_frame

In [ ]:
def doly(x):
    
    gh = x['ID'].count() / frame_size[0]
    print(gh)
    return gh

asu_frame.groupby(['Пол', 'Статус']).apply(doly)

In [ ]:
asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()

In [ ]:
old_year = datetime.datetime.strptime('1986', '%Y')
asu_frame.loc[(asu_frame['birth_year'] < old_year) & (asu_frame['СрБаллАттестата'] > 20)]

In [ ]:
old_year = datetime.datetime.strptime('2004', "%Y")


In [ ]:
asu_frame.groupby('Код_группы')['ID'].count().max()

# Очистка первичного фрейма

In [ ]:
clean_frame = pd.DataFrame()

In [ ]:
clean_frame = asu_frame.copy()

Колонка иностранный язык

In [ ]:
clean_frame.loc[clean_frame['Изучаемый_Язык'].isnull(), 'Изучаемый_Язык'] = '-'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Англиийский', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Английский)', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Немецкий)', 'Изучаемый_Язык'] = 'Немецкий язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Русский язык', 'Изучаемый_Язык'] = '-'
clean_frame.groupby('Изучаемый_Язык')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Пол'].isnull(), 'Пол'] = '-'
clean_frame.groupby('Пол')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Уч_Заведение'].isnull(), 'Уч_Заведение'] = '-'
clean_frame.groupby('Уч_Заведение')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Где_Находится_УЗ'].isnull(), 'Где_Находится_УЗ'] = '-'

In [ ]:
clean_frame.loc[clean_frame['Пособие'].isnull(), 'Пособие'] = 0
clean_frame.groupby('Пособие')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Страна_ПП'].isnull(), 'Страна_ПП'] = '-'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Киргизия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызстан', 'Страна_ПП'] = 'Кыргызская Республика'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан ВКО', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан Респ', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан респ', 'Страна_ПП'] = 'Республика Казахстан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан', 'Страна_ПП'] = 'Республика Таджикистан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан Респ', 'Страна_ПП'] = 'Республика Таджикистан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'РОССИЯ', 'Страна_ПП'] = 'Россия'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Росссия', 'Страна_ПП'] = 'Россия'

russia_city = ['Барнаул', 'г. Барнаул' , 'Славгород', 'Рубцовск', 'Г.Барнаул', 'Г. Новоалтайск', 'Бийск', 'Белокуриха', 'Барнаул г', 'Новоалтайск', 'Благовещенка РП', 'с. Подсосново', 'Среднекрасилово с']
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Город_ПП'].isin(russia_city)), ['Страна_ПП']] = 'Россия'
turkmen_city = ['Туркменабат']
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Город_ПП'].isin(turkmen_city)), ['Страна_ПП']] = 'Туркменистан'

clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Страна_Родители'] == 'Россия'), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Страна_Родители'] == 'Республика Казахстан'), 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Страна_Родители'] == 'Республика Таджикистан'), 'Страна_ПП'] = 'Республика Таджикистан'

clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Где_Находится_УЗ'].str.contains('россия', case=False)), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Где_Находится_УЗ'].str.contains('барнаул', case=False)), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Где_Находится_УЗ'].str.contains('москва', case=False)), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Где_Находится_УЗ'].str.contains('рубцовск', case=False)), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Где_Находится_УЗ'].str.contains('славгород', case=False)), 'Страна_ПП'] = 'Россия'

clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Уч_Заведение'].str.contains('АлтГУ', case=False)), 'Страна_ПП'] = 'Россия'
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Уч_Заведение'].str.contains('островского', case=False)), 'Страна_ПП'] = 'Республика Казахстан'
russia_school = ['ГКУ СОШ №9', 'ГОУ лицей №3', 'КГУ "СОШ №3"']
clean_frame.loc[(clean_frame['Страна_ПП'] == '-') & (clean_frame['Уч_Заведение'].isin(russia_school)), ['Страна_ПП']] = 'Россия'

clean_frame.groupby('Страна_ПП')['ID'].count()


In [ ]:
clean_frame.loc[clean_frame['Регион_ПП'].isnull(), 'Регион_ПП'] = '-'

In [ ]:
clean_frame.loc[clean_frame['Город_ПП'].isnull(), 'Город_ПП'] = '-'

In [ ]:
clean_frame.loc[clean_frame['Общежитие'].isnull(), 'Общежитие'] = 0
clean_frame.groupby('Общежитие')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Страна_Родители'].isnull(), 'Страна_Родители'] = '-'

clean_frame.loc[clean_frame['Страна_Родители'] == 'КНР', 'Страна_Родители'] = 'Китай'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Кыргызстан', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Киргизия', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Таджикистан', 'Страна_Родители'] = 'Республика Таджикистан'

clean_frame.loc[(clean_frame['Страна_Родители'] == '-') & (clean_frame['Страна_ПП'] != '-'), 'Страна_Родители'] = clean_frame.loc[(clean_frame['Страна_Родители'] == '-') & (clean_frame['Страна_ПП'] != '-'), 'Страна_ПП']

clean_frame.groupby('Страна_Родители')['ID'].count()

In [ ]:
clean_frame.loc[clean_frame['Опекунство'].isnull(), 'Опекунство'] = 0
clean_frame.groupby('Опекунство')['ID'].count()

In [ ]:

clean_frame.loc[(clean_frame['Село'].isnull()) & (~clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 1
clean_frame.loc[(clean_frame['Село'].isnull()) & (clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 0
clean_frame.loc[(clean_frame['Село'].isnull())]
clean_frame.groupby('Село')['ID'].count()

In [ ]:

clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (~clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 1
clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 0
clean_frame.loc[(clean_frame['Иностранец'].isnull())]
clean_frame.groupby('Иностранец')['ID'].count()

In [ ]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]

In [ ]:
clean_frame.loc[(clean_frame['СрБаллАттестата'].isnull())]
big_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'ID'].to_list()
small_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'ID'].to_list()
clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] / 100
#clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'СрБаллАттестата'] = clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'СрБаллАттестата'] * 10
clean_frame.loc[clean_frame['ID'].isin(big_atestat), 'СрБаллАттестата']




In [ ]:
clean_frame.loc[(clean_frame['Статус'].isnull())]

In [ ]:
clean_frame['СрБаллАттестата'].describe()

In [ ]:
asu_frame.columns

In [ ]:
columns_to_csv = ['Год_Поступления', 'Пол', 'Основания', 'Изучаемый_Язык', 'Дата_Рождения', 'Страна_ПП', 'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance']

In [ ]:
clean_frame.columns[clean_frame.isna().any()].tolist()

In [ ]:
clean_frame.loc[:, ['Основания', 'male', 'female']].head(10)

# Составление справочников кодов для текстовых колонок

In [ ]:
input_train = clean_frame.head(10).copy()
input_train.head(2)

In [ ]:
clean_frame.info()

In [ ]:
# проверка полноты формирования спика колонок по форматам
non_digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].count()
digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].count()
clean_frame.shape[1] - (non_digit_columns_num + digit_columns_num)

In [ ]:
# список не числовых колонок
non_digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].index.to_list()
non_digit_columns

In [ ]:
# список числовых колонок

digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].index.to_list()
digit_columns

In [ ]:
columns_to_code = {
    'Основания': 'basis',
    'Изучаемый_Язык': 'language',    
    'Страна_ПП': 'country',
    'Регион_ПП': 'region',
    'Город_ПП': 'city',
    'Страна_Родители': 'parents_country',
}

In [ ]:
def code_dict(column_name: str):
    code_frame = clean_frame.groupby(column_name)['ID'].count()
    code_list = code_frame.index.to_list()
    target_dict = {}
    for num, name in enumerate(code_list):
        target_dict[name] = num
    return target_dict

In [ ]:
def test(x, arg):
    mes = f'chore - {x}, change to {arg[x]}'
    print(mes)

#input_train['Изучаемый_Язык'].apply(test, args=({'Английский язык': 'o', '-': 'g'},))

In [ ]:
# Функция для преобразования слова в код по заданному правилу

def get_code(x: pd.Series, code_dict) -> pd.Series:    
    return code_dict[x]



In [ ]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict
    input_train[columns_to_code[name]] = input_train[name].apply(get_code, args=(target_dict,))

input_train.head(3)

In [ ]:
list(columns_to_code.values())

In [ ]:
columns_to_model = digit_columns + list(columns_to_code.values())
columns_to_model

In [ ]:
input_train.loc[:, columns_to_model].to_csv('data/clean.csv', index_label='ind')

In [ ]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict    
    clean_frame[columns_to_code[name]] = clean_frame[name].apply(get_code, args=(code_dict(name),))

clean_frame.head(3)

In [ ]:
frame_to_csv = clean_frame.copy()

In [ ]:

frame_to_csv.loc[:, columns_to_model].to_csv('data/clean_frame.csv', index_label='ind')

In [ ]:
import yaml
with open('data/codes.yaml', 'w') as file:
    yaml.dump(codes_rule, file)